In [65]:

import pandas as pd
import numpy as np
import re
import math
import os
import subprocess

# Text and feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer

# Evaluation and tuning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_curve, auc)

# Classifier
from sklearn.naive_bayes import GaussianNB

# Text cleaning & stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import torch
import torch.nn

[nltk_data] Downloading package stopwords to /Users/dwika/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
from simple_mlp import MLPWithLayerNorm
import TextDataset

In [67]:
from torch.utils.data import random_split, DataLoader

# Step 1: Create the full dataset
full_dataset = TextDataset.TextDatasetTFIDF('datasets/keras.csv')

# Step 2: Define train/val split sizes
val_ratio = 0.2
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size

# Step 3: Split the dataset
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Step 4: Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

Preprocessed DataFrame:
                                                   text  sentiment
361  How to represent 28x1x1 output dense layer?. !...          0
158  Accessing internal states. Hey guys, cool proj...          0
477  docs correction: https://keras.io/activations/...          0
517  SimpleRNN - Wrong number of dimensions, expect...          0
275  merge layer documentation and architecture  . ...          0


In [68]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score

In [69]:
model = MLPWithLayerNorm(input_dim=1000)  # TF-IDF has 1000 features
criterion = nn.CrossEntropyLoss()  # You can add weights here if needed
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Lowered LR


In [70]:
accuracy_scores=[]
f1_scores = []
precision_scores = []
recall_scores = []
auc_scores=[]
epochs=50
# ========== Training Loop ==========
for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation
    model.eval()
    all_preds, all_probs, all_targets = [], [], []

    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_targets.extend(batch_y.cpu().numpy())

    # Metrics
    acc = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds, average='binary')
    precision = precision_score(all_targets, all_preds, average='binary')
    recall = recall_score(all_targets, all_preds, average='binary')
    try:
        auc = roc_auc_score(all_targets, all_probs)
        auc_scores.append(auc)
    except ValueError:
        auc = None

    # Save metrics
    accuracy_scores.append(acc)
    f1_scores.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)
 


    # print(f"Epoch [{epoch+1}/{epochs}]")
    # print(f"  Train Loss : {total_train_loss:.4f}")
    # print(f"  Val Acc    : {acc * 100:.2f}%")
    # print(f"  Val F1     : {f1:.4f}")
    # print(f"  Val Prec   : {precision:.4f}")
    # print(f"  Val Recall : {recall:.4f}")
    # print(f"  Val AUC    : {auc:.4f}" if auc is not None else "  Val AUC    : Not computable")
    # print("-" * 40)

# ========== Averages ==========
avg_acc = sum(accuracy_scores) / len(accuracy_scores)
avg_f1 = sum(f1_scores) / len(f1_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_recall = sum(recall_scores) / len(recall_scores)
auc_score=sum(auc_scores) / len(auc_scores)

print("\n===== Average Validation Metrics Across Epochs =====")
print(f"Average Accc     : {avg_acc:.4f}")
print(f"Average F1 Score     : {avg_f1:.4f}")
print(f"Average Precision    : {avg_precision:.4f}")
print(f"Average Recall       : {avg_recall:.4f}")
print(f"Average Auc Score     : {auc_score:.4f}")


===== Average Validation Metrics Across Epochs =====
Average Accc     : 0.8708
Average F1 Score     : 0.5766
Average Precision    : 0.7918
Average Recall       : 0.4562
Average Auc Score     : 0.8785


# Prediction on Different Data Set


In [71]:
inference_dataset=TextDataset.TextDatasetTFIDF("datasets/tensorflow.csv")
inference_loader = DataLoader(inference_dataset, batch_size=64, shuffle=False)

Preprocessed DataFrame:
                                                   text  sentiment
941  Graph_def is invalid at node u'ExpandDims': In...          0
297  stream_executor/platform/mutex.h doesn't compi...          0
271  [Perfomance]Dilated/Atrous Conv implementation...          1
774  Tensorflow 2.0 keras load_model does not resto...          0
420  Multiprocessing for input pipeline . I have as...          0


In [72]:
model.eval()
all_preds = []
all_probs = []
all_targets = []

with torch.no_grad():
    for batch_X, batch_y in inference_loader:
        outputs = model(batch_X)
        probs = torch.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())
        all_targets.extend(batch_y.cpu().numpy())


In [73]:
accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average='binary')
recall = recall_score(all_targets, all_preds, average='binary')
f1 = f1_score(all_targets, all_preds, average='binary')
try:
    auc = roc_auc_score(all_targets, all_probs)
except ValueError:
    auc = None

print(f"Accuracy  : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1 Score  : {f1:.4f}")
print(f"AUC       : {auc:.4f}" if auc is not None else "AUC: Not computable")

Accuracy  : 0.8013
Precision : 0.1600
Recall    : 0.0143
F1 Score  : 0.0263
AUC       : 0.4988


# Trying Tent on Title Body Caffe

In [74]:
import tent
import TextDataset
import importlib
# import batch_data_loader
importlib.reload(TextDataset)  
importlib.reload(tent)
import copy

In [75]:
dataset_tent=TextDataset.TextDatasetTFIDF("datasets/tensorflow.csv")
dataloader_tent= DataLoader(dataset_tent, batch_size=64, shuffle=True)

Preprocessed DataFrame:
                                                   text  sentiment
941  Graph_def is invalid at node u'ExpandDims': In...          0
297  stream_executor/platform/mutex.h doesn't compi...          0
271  [Perfomance]Dilated/Atrous Conv implementation...          1
774  Tensorflow 2.0 keras load_model does not resto...          0
420  Multiprocessing for input pipeline . I have as...          0


In [76]:
model_state = copy.deepcopy(model.state_dict())

In [77]:
model=tent.configureTent(model)
result=tent.Tent(model,dataloader_tent,"keras","mxnet",1)
print(result)
tent.reset(model,model_state)

Adapting with Tent: 100%|██████████| 24/24 [00:00<00:00, 40.19it/s]


📊 Test-Time Adaptation (Tent) Metrics:
  Accuracy : 81.28%
  F1 Score : 0.0071
  Precision: 0.5000
  Recall   : 0.0036
  ROC AUC  : 0.5134
{'iteration': 1, 'name': 'keras->mxnet', 'accuracy': 0.812751677852349, 'precision': 0.5, 'recall': 0.0035842293906810036, 'f1': 0.0071174377224199285, 'auc': 0.5133646472449381}
